# Fundamentos de Apache Spark: RDDS

En este notebook trabajaremos con los RDDs que forma parte del Spark Core.La implementación de Spark Core es un RDD (Resilient Distributed Dataset) que es una colección de datos distribuidos en diferentes nodos del clúster que se procesan en paralelo.

Utilizaremos la API de PySpark, pero los conceptos aplican por igual a todas las APIs (Scala, R, etc)

#### INICIALIZACIÓN DE SPARK EN NOTEBOOKS

In [2]:
# Debemos instalar findspark si no la tenemos
# !pip install findspark

In [3]:
import findspark    # importamos findspark
findspark.init()

import pandas as pd # importamos pandas y pyspark
import pyspark

In [4]:
from pyspark.sql import SparkSession # De la libreria pyspark.sql importamos SparkSession

#### CREAR EL SPARKSESSION Y EL SPARKCONTEXT

In [5]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_training')\
        .getOrCreate()

In [6]:
# Otra forma de crear SparkSession de forma más sencilla
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

### Crear un RDD de una colección

In [7]:
num = [1,2,3,4,5]
num_rdd = sc.parallelize(num)
num_rdd.collect()

[1, 2, 3, 4, 5]

### Transformaciones
* Como sabemmos, las transformaciones son de naturaleza perezosa y no se ejecutaran hasta que se ejecute una Acción sobre ellas
* Intentemos comprender las distintas transformaciones disponobles

#### map
* Esto mapeará su entrada a alguuna salida basada en la función especificada en la función

In [8]:
double_rdd = num_rdd.map(lambda x: x*2)
double_rdd.collect()

[2, 4, 6, 8, 10]

#### filter
* Para filtrar los datos en función de una determinada condición. Intentemos encontrar los números pares de num_rdd

In [9]:
even_rdd = num_rdd.filter(lambda x : x % 2 == 0)
even_rdd.collect()

[2, 4]

#### flatmap
* Esta función reduce los pares de valores clave en función de las claves y una funcion determinada dentro de reduceByKey

In [10]:
pairs = [('a', 5), ('b', 7), ("c", 2),("a", 7),("b", 1),("c", 4)]
pair_rdd = sc.parallelize(pairs)

output = pair_rdd.reduceByKey(lambda x, y: x + y)

result = output.collect()
print(*result, sep='\n')

('b', 8)
('a', 12)
('c', 6)


#### distinct
* Esto devolverá elementos distintos de un RDD

In [11]:
rdd1 = sc.parallelize([10, 11, 10, 11, 12, 11])
dist_rdd = rdd1.distinct()
dist_rdd.collect()

[12, 10, 11]

#### reduceByKey
* Esta función reduce los pares de valores clave en función de las claves y una función determinada dentro de reduceByKey

In [12]:
pairs = [ ("a", 5), ("b", 7), ("c", 2), ("a", 3), ("b", 1), ("c", 4)]
pair_rdd = sc.parallelize(pairs)

output = pair_rdd.reduceByKey(lambda x, y : x + y)

result = output.collect()
print(*result, sep='\n')

('b', 8)
('a', 8)
('c', 6)


#### groupByKey
* Esta función es otra función ByKey que puede operar en un par (clave, valor) RDD pero esto solo agrupará los valores basados en las claves. En otras palabras, esto solo lo realizará el primer paso de reduceByKey

In [14]:
grp_out = pair_rdd.groupByKey()
grp_out.collect()

[('b', <pyspark.resultiterable.ResultIterable at 0x2370891f1c8>),
 ('a', <pyspark.resultiterable.ResultIterable at 0x2370891f508>),
 ('c', <pyspark.resultiterable.ResultIterable at 0x23707ef1ec8>)]

#### sortByKey
* Esta función realizará la clasificación en un par (clave, valor) RDD basado en las claves. De forma predeterminada, la clasificación se realizará en orden ascendente

In [15]:
pairs =  [ ("a", 5), ("d", 7), ("c", 2), ("b",3)]
raw_rdd = sc.parallelize(pairs)

sortkey_rdd = raw_rdd.sortByKey()
result = sortkey_rdd.collect()
print(result, sep='\n')

# Para clasificar en orden descendente, determinar "ascending=Flase"

('a', 5)
('b', 3)
('c', 2)
('d', 7)


#### sortBy
* es una función mas generalizada para ordenar

In [17]:
#Create RDD
pairs = [("a", 5, 10), ("d", 7, 12), ("c", 2, 11), ("b", 3, 9)]
raw_rdd = sc.parallelize(pairs)

# Let's try to do the sorting based on the 3rd element of the tuple
sort_out = raw_rdd.sortBy(lambda x : x[2])
result = sort_out.collect()
print(result, sep ='\n')

[('b', 3, 9), ('a', 5, 10), ('c', 2, 11), ('d', 7, 12)]


### Acciones
* Las acciones son operaciones en RDD que se ejecutan inmediatamente. Mientras que las transformaciones devuelven otro RDD, las acciones devuelven estructuras de datos nativas





#### count
* esto contará el número de elementos en el RDD dado

In [18]:
num = sc.parallelize([1,2,3,4,2])
num.count()

5

#### first
* Esto devolverá el primer elemento del RDD dado

In [19]:
num.first()

1

#### collect
* Esto devolverá todos los elementos para el RDD dado

In [20]:
num.collect()

[1, 2, 3, 4, 2]

**No debemos utiliza la operación de collect mientras trabajamos con grandes conjuntos de datos.** Porque devolverá todos los datos que se distribuyen entre los diferentes trabajadores del cluster a un controlador. Todos los datos viajarán a través de la red del trabajador al conductor y también el conducto necesitaria almacenar todos los datos. Esto obstaculizará el rendimiento de su aplicación

#### Take
* Esto devolverá el número de elementos especificados

In [13]:
num.take()

AttributeError: 'list' object has no attribute 'take'